In [104]:
import requests
import email
from imapclient import IMAPClient
from bs4 import BeautifulSoup
import time
import json

In [24]:
google_form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdvsBIQTkYz7MUq1PCpVgnROiZnbNFoiGGs58rFcM0lqBntkw/viewform?usp=sf_link"
email_address = "fthenosyd@gmail.com"
email_password = "yuvrnxoilswwwife"

In [107]:
budget = json.loads(open("budget.json").read())
print(budget)

{'income': '1300', 'budget': {'house': '340', 'gas': '100', 'supermarket': '150', 'entertainment': '150', 'efood': '50', 'bills': '70', 'others': '100', 'total': '960'}, 'savings': '340'}


In [96]:
USER_KEY = "u7rpsco3e2gxpbm2or6g7cpwhwom24"
API_TOKEN = "atvoawk91m3gvjo1tynapr65ga3a49"

def send_budget_notification(payload):
    response = requests.post('https://api.pushover.net/1/messages.json', data=payload)
    
    if response.status_code == 200:
        print("Notification sent.")
    else:
        print(f"Error sending notification. Status code: {response.status_code}")

In [163]:
def read_daily_expenses_email():
        # Search for unread emails with a specific subject
        while True:
            with IMAPClient(host='imap.gmail.com', use_uid=True, ssl=True) as client:
                client.login(email_address, email_password)
                client.select_folder('INBOX')
                messages = client.search(['UNSEEN', 'SUBJECT', 'Budget response'])

                if messages:
                    for msg_id in messages:
                        amounts = dict()
                        response_data = client.fetch([msg_id], 'BODY[TEXT]')[msg_id][b'BODY[TEXT]']
                        msg = email.message_from_bytes(response_data)
                        soup = BeautifulSoup(msg.get_payload(decode=True), 'html.parser')
                        # Parse the email content to extract the submitted expense
                        spans = soup.find_all('span')
                        
                        amounts['rent'] = spans[0].text
                        amounts['car_installement'] = spans[1].text
                        amounts['gasoline'] = spans[2].text
                        amounts['supermarket'] = spans[3].text
                        amounts['entertainment'] = spans[4].text
                        amounts['efood'] = spans[5].text
                        amounts['bills'] = spans[5].text
                        amounts['others'] = spans[6].text
                        # Mark the email as read
                        client.delete_messages(msg_id)
                        break
                    return amounts
                else:
                    #client.logout()
                    time.sleep(5)


In [164]:
send_budget_notification(payload = {
        'message': 'Εισαγωγή των σημερινών εξόδων',
        'token': API_TOKEN,
        'user': USER_KEY,
        'title': 'Budget Alert',
        'url': google_form_url,
        'url_title': "Enter your today's expanes",
    })


Notification sent.


In [165]:
daily_expenses = read_daily_expenses_email()

print(f"Today's expenses: {daily_expenses}")

Today's expenses: {'rent': '0', 'car_installement': '0', 'gasoline': '0', 'supermarket': '0', 'entertainment': '0', 'efood': '10', 'bills': '10', 'others': '10.5'}


Replace special characters

In [166]:
try:
    daily_expenses['gasoline'] = daily_expenses['gasoline'].replace("\r", "")
    try:
        daily_expenses['gasoline'] = daily_expenses['gasoline'].replace("\n", "")
        try:
            daily_expenses['gasoline'] = daily_expenses['gasoline'].replace("=", "")
        except Exception:
            pass
    except Exception:
        pass
except Exception:
    pass

print(daily_expenses)

{'rent': '0', 'car_installement': '0', 'gasoline': '0', 'supermarket': '0', 'entertainment': '0', 'efood': '10', 'bills': '10', 'others': '10.5'}


Replace letter if any and turn the values into float numbers

In [167]:
import re
for key in daily_expenses.keys():
    daily_expenses[key] = re.sub(r'[^\d]', '', daily_expenses[key])
    daily_expenses[key] = float(daily_expenses[key])
print(daily_expenses)

{'rent': 0.0, 'car_installement': 0.0, 'gasoline': 0.0, 'supermarket': 0.0, 'entertainment': 0.0, 'efood': 10.0, 'bills': 10.0, 'others': 105.0}
